In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import load_model

In [10]:
yolo_model = load_model("/Desktop/DeepLearning/saved_model.pb", compile=False)

OSError: SavedModel file does not exist at: /Desktop/DeepLearning/saved_model.pb/{saved_model.pbtxt|saved_model.pb}